# Membrane Segmentation
This notebook demonstrates how to segment cell membranes from fluorescence microscopy images.

## Import required libraries

In [1]:
import numpy as np
from skimage import io, filters, segmentation
from skimage.morphology import remove_small_objects
import matplotlib.pyplot as plt

## Load and display the image

In [2]:
image = io.imread('membrane_image.tif')
plt.imshow(image, cmap='gray')
plt.axis('off')
plt.savefig('original.png')
plt.close()

TiffFileError: not a TIFF file b'\x89PNG'

## Preprocess image using Gaussian smoothing

In [ ]:
smoothed = filters.gaussian(image, sigma=1)
plt.imshow(smoothed, cmap='gray')
plt.axis('off')
plt.savefig('smoothed.png')
plt.close()

## Threshold the image to separate membranes

In [ ]:
threshold = filters.threshold_otsu(smoothed)
binary = smoothed > threshold
plt.imshow(binary, cmap='gray')
plt.axis('off')
plt.savefig('binary.png')
plt.close()

## Segment cells using watershed

In [ ]:
distance = segmentation.watershed(-smoothed, markers=100, mask=~binary)
labels = remove_small_objects(distance > 0, min_size=100)

plt.imshow(labels, cmap='nipy_spectral')
plt.axis('off')
plt.savefig('segmented.png')
plt.close()

io.imsave('labels.png', (labels * 255 / labels.max()).astype(np.uint8))